In [1]:
import zipfile
import pandas as pd

def read_zipped_csv(zipfilename, filename):
    """
    Open up a zipfile, and read out csv data from the speficied filename
    into a pandas dataframe.

    Parameters
    ----------
    zipfilename:
        Source zip file on disk.
    filename:
        File within the zip containing the data

    Returns
    -------
    A Pandas DataFrame.
    """
    # target path, wrapped in an object
    with zipfile.ZipFile(zipfilename) as archive:
        with archive.open(filename) as file:
            return pd.read_csv(file)

loans = read_zipped_csv('./lending-club-loan-data.zip', 'loan.csv')

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2802: DtypeWarning: Columns (19,55) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [2]:
import sqlite3
import zipfile
import pandas as pd

def read_zipped_sqlite(zipfilename, filename, query):
    """
    Open up a zipfile, and read out table data from the speficied filename
    into a pandas dataframe.

    Parameters
    ----------
    zipfilename:
        Source zip file on disk.
    filename:
        File within the zip containing the data
    tablename:
        Read this entire table.

    Returns
    -------
    A Pandas DataFrame.
    """
    # target path, wrapped in an object
    with zipfile.ZipFile(zipfilename) as archive:
        archive.extract(filename)
        with sqlite3.connect(filename) as connection:
            return pd.read_sql(query, connection)

loans = read_zipped_sqlite('./lending-club-loan-data.zip', 'database.sqlite', 'select * from loan')

In [10]:
loans.columns.values

array(['index', 'id', 'member_id', 'loan_amnt', 'funded_amnt',
       'funded_amnt_inv', 'term', 'int_rate', 'installment', 'grade',
       'sub_grade', 'emp_title', 'emp_length', 'home_ownership',
       'annual_inc', 'verification_status', 'issue_d', 'loan_status',
       'pymnt_plan', 'url', 'desc', 'purpose', 'title', 'zip_code',
       'addr_state', 'dti', 'delinq_2yrs', 'earliest_cr_line',
       'inq_last_6mths', 'mths_since_last_delinq',
       'mths_since_last_record', 'open_acc', 'pub_rec', 'revol_bal',
       'revol_util', 'total_acc', 'initial_list_status', 'out_prncp',
       'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv',
       'total_rec_prncp', 'total_rec_int', 'total_rec_late_fee',
       'recoveries', 'collection_recovery_fee', 'last_pymnt_d',
       'last_pymnt_amnt', 'next_pymnt_d', 'last_credit_pull_d',
       'collections_12_mths_ex_med', 'mths_since_last_major_derog',
       'policy_code', 'application_type', 'annual_inc_joint', 'dti_joint',
       'verificat

In [11]:
loans.iloc[0:1].values

array([[0, '1077501', 1296599.0, 5000.0, 5000.0, 4975.0, ' 36 months',
        ' 10.65%', 162.87, 'B', 'B2', None, '10+ years', 'RENT', 24000.0,
        'Verified', 'Dec-2011', 'Fully Paid', 'n',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=1077501',
        '  Borrower added on 12/22/11 > I need to upgrade my business technologies.<br>',
        'credit_card', 'Computer', '860xx', 'AZ', 27.65, 0.0, 'Jan-1985',
        1.0, nan, nan, 3.0, 0.0, 13648.0, '83.7%', 9.0, 'f', 0.0, 0.0,
        5861.071414249001, 5831.78, 5000.0, 861.07, 0.0, 0.0, 0.0,
        'Jan-2015', 171.62, None, 'Jan-2016', 0.0, nan, 1.0, 'INDIVIDUAL',
        nan, nan, None, 0.0, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan]], dtype=object)

In [12]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import numpy as np

def transform_categorical(column_of_data):
    """
    Given a column of data (a vector, list or array) from a tabular data source,
    transform it to a one-hot encoded matrix suitable for machine learning.
    
    Parameters
    ----------
    column_of_data:
        Source data column samples to be transformed.
        
    Returns
    -------
    A 2d numpy array, with a column for each new dimension, and a row for each
    sample passed.
    """
    handle_none = list(map(str, column_of_data))
    
    labels = LabelEncoder()
    encoded_labels = labels.fit_transform(handle_none)
    
    one_hot = OneHotEncoder()
    return one_hot.fit_transform(encoded_labels.reshape(-1, 1)).astype(np.float32)

In [13]:
loans.loan_status[0]

'Fully Paid'

In [14]:
statuses = transform_categorical(loans.loan_status)

In [15]:
statuses[0].todense()

matrix([[ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.]], dtype=float32)

In [16]:
from sklearn.preprocessing import StandardScaler
import numpy as np

def transform_numerical(column_of_data):
    """
    Given a column of data (a vector, list or array) from a tabular data source,
    transform it by scaling the values.
    
    Parameters
    ----------
    column_of_data:
        Source data column samples to be transformed.
        
    Returns
    -------
    A 1d numpy array, with a scaled set of values
    """
    as_feature_array = np.array(column_of_data).reshape(-1, 1)
    
    zeroed = np.nan_to_num(as_feature_array)
    
    scaled = StandardScaler()
    return scaled.fit_transform(zeroed)

In [17]:
loans.loan_amnt[0]

5000.0

In [18]:
amounts = transform_numerical(loans.loan_amnt)

In [19]:
amounts[0]

array([-1.15645878])

In [25]:
loans.emp_length.unique()

array(['10+ years', '< 1 year', '1 year', '3 years', '8 years', '9 years',
       '4 years', '5 years', '6 years', '2 years', '7 years', 'n/a', None], dtype=object)

In [32]:
from sklearn.pipeline import make_pipeline
from importlib import reload
import ColumnSet
reload(ColumnSet)

pipe = make_pipeline(
    ColumnSet.PercentageColumn(['int_rate', 'revol_util']),
    ColumnSet.ClassifierColumnSet(['home_ownership', 'emp_length'], 
                                  ['loan_amnt', 'int_rate', 'revol_util', 'total_pymnt', 'inq_last_6mths', 'dti',
                                  'installment', 'annual_inc'],
                                  'loan_status')
)


(X, Y) = pipe.fit_transform(loans.loc[loans.loan_status.isin(['Current', 'Default'])])

In [30]:
X.shape

(603000, 23)

In [31]:
Y.shape

(603000, 2)